In [25]:
import math
import random
import time
from readinput import read_multiple_test_cases
file_path = 'input.txt'
test_cases = read_multiple_test_cases(file_path)

In [26]:
def simulated_annealing(n, k, distance_matrix, initial_temperature, cooling_rate, max_iterations):
    
    def calculate_cost(route, distance_matrix):
        cost = distance_matrix[0][route[0]]
        for i in range(len(route) - 1):
            cost += distance_matrix[route[i]][route[i + 1]]
        cost += distance_matrix[route[-1]][0]
        return cost

    def generate_neighbor(route):
        i, j = sorted(random.sample(range(len(route)), 2))
        neighbor = route[:]
        neighbor[i:j+1] = reversed(neighbor[i:j+1])
        return neighbor
    
    def is_valid_route(route, n, k):
        # Kiểm tra ràng buộc đón trước trả và sức chứa
        current_load = 0
        position = {val: idx for idx, val in enumerate(route)}
        for i in range(1, n + 1):
            if position[i] > position[i + n]:  # Đón sau trả
                return False
        for point in route:
            if point <= n:
                current_load += 1
            else:
                current_load -= 1
            if current_load > k:
                return False
        return True
    
    def generate_random_valid_route(n, k):
        while True:
            route = list(range(1, 2 * n + 1))
            random.shuffle(route)
            if is_valid_route(route, n, k):
                return route

    current_route = generate_random_valid_route(n, k)
    current_cost = calculate_cost(current_route, distance_matrix)
    best_route = current_route
    best_cost = current_cost
    temperature = initial_temperature

    for iteration in range(max_iterations):
        neighbor = generate_neighbor(current_route)
        if not is_valid_route(neighbor, n, k):
            continue
        
        neighbor_cost = calculate_cost(neighbor, distance_matrix)
        
        if neighbor_cost < current_cost:
            current_route = neighbor
            current_cost = neighbor_cost
            
        else:
            delta = neighbor_cost - current_cost
            probability = math.exp(-delta / temperature)
            if random.random() < probability:
                current_route = neighbor
                current_cost = neighbor_cost
                
        if current_cost < best_cost:
            best_route = current_route
            best_cost = current_cost
            
        temperature *= cooling_rate
        if temperature < 1e-5:
            break

    return best_route, best_cost

In [27]:
for idx, (n, k, distance_matrix) in enumerate(test_cases, 1):
    print(f"Test case {idx}:")
    print(f"n = {n}, k = {k}")
    print(f"Cost matrix:")
    for row in distance_matrix:
        print(row)
    print()
    start_time = time.time()
    best_route, best_cost = simulated_annealing(n, k, distance_matrix, 1000000, 0.995, 50000)
    end_time = time.time()
    print(f"Min cost: {best_cost}")
    print(f"Best route: {' '.join(map(str, best_route))}")
    print(f"Calculation time: {end_time - start_time:.2f} seconds")
    print()

Test case 1:
n = 3, k = 2
Cost matrix:
[0, 8, 5, 1, 10, 5, 9]
[9, 0, 5, 6, 6, 2, 8]
[2, 2, 0, 3, 8, 7, 2]
[5, 3, 4, 0, 3, 2, 7]
[9, 6, 8, 7, 0, 9, 10]
[3, 8, 10, 6, 5, 0, 2]
[3, 4, 4, 5, 2, 2, 0]

Min cost: 25
Best route: 3 1 4 2 6 5
Calculation time: 0.07 seconds

Test case 2:
n = 5, k = 3
Cost matrix:
[0, 2, 3, 4, 2, 3, 4, 5, 3, 8, 7]
[5, 0, 1, 4, 6, 1, 4, 2, 9, 8, 2]
[1, 2, 0, 4, 2, 3, 4, 5, 3, 6, 2]
[2, 2, 3, 0, 2, 3, 4, 5, 3, 8, 4]
[5, 2, 3, 4, 0, 3, 4, 5, 3, 9, 7]
[1, 2, 3, 1, 2, 0, 4, 2, 2, 8, 2]
[2, 2, 3, 4, 2, 3, 0, 5, 3, 2, 7]
[3, 2, 3, 6, 2, 3, 4, 0, 3, 1, 1]
[5, 2, 3, 4, 2, 3, 4, 5, 0, 8, 7]
[1, 2, 3, 5, 2, 3, 4, 5, 3, 0, 9]
[8, 2, 3, 4, 2, 3, 4, 5, 3, 2, 0]

Min cost: 25
Best route: 1 5 3 8 4 6 2 10 7 9
Calculation time: 0.09 seconds

Test case 3:
n = 5, k = 3
Cost matrix:
[0, 5, 8, 11, 12, 8, 3, 3, 7, 5, 5]
[5, 0, 3, 5, 7, 5, 3, 4, 2, 2, 2]
[8, 3, 0, 7, 8, 8, 5, 7, 1, 6, 5]
[11, 5, 7, 0, 1, 5, 9, 8, 6, 5, 6]
[12, 7, 8, 1, 0, 6, 10, 10, 7, 7, 7]
[8, 5, 8, 5, 6, 0, 8, 5, 7,